In [1]:
import os
#set your environment variable for SSL certificate
certi_path = r"C:\Users\bertr\AppData\Local\Programs\Python\Python39\Lib\site-packages\pip\_vendor\certifi\cacert.pem"
os.environ['REQUESTS_CA_BUNDLE'] = certi_path
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import random
import time
import json
import pandas as pd

In [2]:
def scroll(driver, i):
        scroll_delta = int(250)
        scroll_delta += 140*i
        driver.execute_script("window.scrollBy(0, "+ str(scroll_delta) + ")")

In [3]:
def initialize_driver(headers_list, proxy_list):
    options = Options()
    #select a random user-agent from the list
    user_agent = random.choice(headers_list)["User-Agent"]
    options.add_argument(f"user-agent={user_agent}")
    
    #select a random proxy from the list
    proxy = random.choice(proxy_list)
    if proxy:
        options.add_argument(f"--proxy-server={proxy}")
    
    #add some common options
    #options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")

    #initialize Chrome WebDriver with the specified options
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    #set implicit wait of 10sec
    driver.implicitly_wait(10)

    return driver

In [4]:
def MainPage(driver, url):
    driver.get(url)
    time.sleep(3)

In [5]:
def nbOffers(driver):
    try:
        search_results = driver.find_elements(By.CSS_SELECTOR, 'li[data-testid="search-results-list-item-wrapper"]')
        search_results_count = len(search_results)
        return search_results_count

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0  # Or handle the exception as needed

In [6]:
def nbOffers_tot(driver):
    try:
        search_result = int(driver.find_element(By.CSS_SELECTOR, 'div[data-testid="jobs-search-results-count"]').text)
        return search_result

    except Exception as e:
        print("An error occurred in NB_OFFER TOTAL:", str(e))
        return 0  # Or handle the exception as needed

In [7]:
def Click(driver, pos):
    '''Click on the link'''
    try:
        element = driver.find_elements(By.CSS_SELECTOR, 'li[data-testid="search-results-list-item-wrapper"]')[pos]
        element.click()
        return 1

    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0

In [8]:
def GetText(driver, jobs):
    time.sleep(3)
    try:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        info = soup.find_all('script', {"type": "application/ld+json"})
        if info:
            job = json.loads(str(info[0].string))
            faq = json.loads(str(info[1].string))
            job['FAQPage'] = faq['mainEntity']
            jobs.append(job)
            f = open('job_text.txt', 'a')
            f.write(str(job))
            print(f'write job : {driver.current_url}')
            f.write('\n')
            f.close()
            return 1
        return None
    except Exception as e:
        print(f"Error HTML PARSING: {e}")

In [9]:
def extract_salary_info(salary_data):
    if pd.isna(salary_data):
        return [None, None, None, None]
    currency = salary_data.get('currency', None)
    value = salary_data.get('value', {})
    min_value = value.get('minValue', None)
    max_value = value.get('maxValue', None)
    unit_text = value.get('unitText', None)
    return [min_value, max_value, currency, unit_text]

In [10]:
driver = initialize_driver([
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Ch-Ua": "\"Chromium\";v=\"92\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"92\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15"
    }
], [False])

In [11]:
MainPage(driver, r"https://www.welcometothejungle.com/fr/jobs?page=1&configure%5Bfilters%5D=website.reference%3Awttj_fr&configure%5BhitsPerPage%5D=30&aroundQuery=France&refinementList%5Boffice.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Stage&query=%22data%20analyst%22&range%5Bexperience_level_minimum%5D%5Bmin%5D=0&range%5Bexperience_level_minimum%5D%5Bmax%5D=1")

In [12]:
nb_offer_page = nbOffers(driver)

In [13]:
nb_toto = nbOffers_tot(driver)

In [14]:
Click(driver, 0)

1

In [15]:
nb_page = nb_toto / nb_offer_page
if nb_toto % nb_offer_page != 0:
    nb_page = nb_page + 1
    nb_page = int(nb_page)

In [29]:
nb_page

3

In [30]:
jobs = []
element = driver.find_elements(By.TAG_NAME, "nav")[1]
element_a = driver.find_elements(By.CSS_SELECTOR, 'a[aria-current="false"]')
for a in range(0, nb_page - 1):
    nb_offer_page = nbOffers(driver)
    for i in range(0, nb_offer_page):
        Click(driver, i)
        GetText(driver, jobs)
        driver.back()
    time.sleep(3)
    element = driver.find_elements(By.TAG_NAME, "nav")[1]
    element_a = driver.find_elements(By.CSS_SELECTOR, 'a[aria-current="false"]')
    element_a[a].click()

write job : https://www.welcometothejungle.com/fr/companies/sparted/jobs/customer-data-analyst-stage-juillet-2024-h-f_paris?q=5af085153a6f6b2639a4f37149b3c9b9&o=d7eed743-ba03-49b6-bec5-7c13e5f757d6
write job : https://www.welcometothejungle.com/fr/companies/caisse-d-epargne-languedoc-roussillon/jobs/data-analyst-direction-controle-de-gestion-montpellier-f-h_montpellier?q=5af085153a6f6b2639a4f37149b3c9b9&o=ad8bdfce-1a00-4cf5-a2a0-bb64299c6843
Error HTML PARSING: 'charmap' codec can't encode character '\u25cf' in position 1883: character maps to <undefined>
write job : https://www.welcometothejungle.com/fr/companies/bouygues-telecom/jobs/data-analyst-financier-f-h_meudon_BT_Jr99y38?q=5af085153a6f6b2639a4f37149b3c9b9&o=ebeb1a27-6c44-4d66-a263-52c65fac12c6
write job : https://www.welcometothejungle.com/fr/companies/bsport-1/jobs/customer-data-migration-specialist-internship_barcelona?q=5af085153a6f6b2639a4f37149b3c9b9&o=09e94bc7-6688-491a-8467-9eec8dbcec12
Error HTML PARSING: 'charmap' cod

In [31]:
jobs

[{'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'baseSalary': {'@type': 'MonetaryAmount',
   'currency': 'EUR',
   'value': {'@type': 'QuantitativeValue',
    'minValue': 1000,
    'maxValue': 1800,
    'unitText': 'YEARLY'}},
  'datePosted': '2024-03-25T15:00:02Z',
  'description': '<p><strong>Le Customer Data Analyst joue un rôle essentiel dans la compréhension du comportement et des besoins des clients d’une startup SaaS, en utilisant des données pour guider les décisions stratégiques et les actions opérationnelles visant à améliorer l’expérience client et à stimuler la croissance de l’entreprise.</strong></p><ul><li><p>Analyse des données client : Collecter, analyser et interpréter les données client provenant de différentes sources telles que les interactions avec le produit, les comportements d’utilisation, les rétroactions des clients.</p></li><li><p>Mise à disposition de tableaux de bord et rapports : Concevoir et maintenir des tableaux de bord et des rapports rég

In [32]:
df = pd.DataFrame(jobs)
df.head()

,@context,@type,baseSalary,datePosted,description,employmentType,educationRequirements,experienceRequirements,hiringOrganization,industry,jobLocation,qualifications,title,validThrough,FAQPage
0,http://schema.org,JobPosting,"{'@type': 'MonetaryAmount', 'currency': 'EUR',...",2024-03-25T15:00:02Z,<p><strong>Le Customer Data Analyst joue un rô...,INTERN,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'SPARTED', '...","SaaS / Cloud Services, Ressources humaines, Ed...","[{'@type': 'Place', 'address': {'@type': 'Post...",Tu es en étude au sein d’un cursus commercial ...,Customer Data Analyst (Stage - Juillet 2024) -...,2024-06-23T14:00:02.000Z,"[{'@type': 'Question', 'name': 'L'envoi d'un C..."
1,http://schema.org,JobPosting,NaN,2024-03-25T08:44:58Z,<p><strong>Poste et missions</strong></p><p>La...,FULL_TIME,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Caisse D'Ép...","Banque, Assurance, Finance","[{'@type': 'Place', 'address': {'@type': 'Post...",Issu(e) d'une formation Bac +5 types Ecole d'I...,Data Analyst - Direction Contrôle de Gestion -...,2024-06-23T07:44:58.000Z,"[{'@type': 'Question', 'name': 'Le télétravail..."
2,http://schema.org,JobPosting,NaN,2024-03-22T17:41:24Z,"<p>At Le Collectionist (LC), The Data team is ...",INTERN,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Le Collecti...","Hôtellerie, Tourisme, Immobilier particulier","[{'@type': 'Place', 'address': {'@type': 'Post...",Preferred backgroundBac+5 engineering or data ...,Data Analyst (H/F) - 6 months internship - Paris,2024-06-20T16:41:24.000Z,"[{'@type': 'Question', 'name': 'L'envoi d'un C..."
3,http://schema.org,JobPosting,NaN,2024-03-22T10:29:22Z,<p><em><strong>Si dans votre métier vous aimez...,FULL_TIME,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Bouygues Te...","IT / Digital, Objets connectés, Electronique /...","[{'@type': 'Place', 'address': {'@type': 'Post...",À propos de vous :Vous êtes diplômé d'une form...,Data Analyst Financier F/H,2024-06-20T09:29:22.000Z,"[{'@type': 'Question', 'name': 'Le télétravail..."
4,http://schema.org,JobPosting,NaN,2024-03-22T09:10:03Z,<p>We’re actively seeking a driven student wit...,INTERN,NaN,{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'BSPORT SOLU...","Application mobile, SaaS / Cloud Services","[{'@type': 'Place', 'address': {'@type': 'Post...",You will be a good fit to join us if you:Techn...,Data Analyst & Operations - Internship,2024-06-20T08:10:03.000Z,"[{'@type': 'Question', 'name': 'L'envoi d'un C..."


In [33]:
df[['minSalary', 'maxSalary', 'currency', 'salaryUnit']] = df['baseSalary'].apply(lambda x: pd.Series(extract_salary_info(x)))

In [34]:
df

,@context,@type,baseSalary,datePosted,description,employmentType,educationRequirements,experienceRequirements,hiringOrganization,industry,jobLocation,qualifications,title,validThrough,FAQPage,minSalary,maxSalary,currency,salaryUnit
0,http://schema.org,JobPosting,"{'@type': 'MonetaryAmount', 'currency': 'EUR',...",2024-03-25T15:00:02Z,<p><strong>Le Customer Data Analyst joue un rô...,INTERN,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'SPARTED', '...","SaaS / Cloud Services, Ressources humaines, Ed...","[{'@type': 'Place', 'address': {'@type': 'Post...",Tu es en étude au sein d’un cursus commercial ...,Customer Data Analyst (Stage - Juillet 2024) -...,2024-06-23T14:00:02.000Z,"[{'@type': 'Question', 'name': 'L'envoi d'un C...",1000.0,1800.0,EUR,YEARLY
1,http://schema.org,JobPosting,NaN,2024-03-25T08:44:58Z,<p><strong>Poste et missions</strong></p><p>La...,FULL_TIME,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Caisse D'Ép...","Banque, Assurance, Finance","[{'@type': 'Place', 'address': {'@type': 'Post...",Issu(e) d'une formation Bac +5 types Ecole d'I...,Data Analyst - Direction Contrôle de Gestion -...,2024-06-23T07:44:58.000Z,"[{'@type': 'Question', 'name': 'Le télétravail...",NaN,NaN,None,None
2,http://schema.org,JobPosting,NaN,2024-03-22T17:41:24Z,"<p>At Le Collectionist (LC), The Data team is ...",INTERN,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Le Collecti...","Hôtellerie, Tourisme, Immobilier particulier","[{'@type': 'Place', 'address': {'@type': 'Post...",Preferred backgroundBac+5 engineering or data ...,Data Analyst (H/F) - 6 months internship - Paris,2024-06-20T16:41:24.000Z,"[{'@type': 'Question', 'name': 'L'envoi d'un C...",NaN,NaN,None,None
3,http://schema.org,JobPosting,NaN,2024-03-22T10:29:22Z,<p><em><strong>Si dans votre métier vous aimez...,FULL_TIME,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Bouygues Te...","IT / Digital, Objets connectés, Electronique /...","[{'@type': 'Place', 'address': {'@type': 'Post...",À propos de vous :Vous êtes diplômé d'une form...,Data Analyst Financier F/H,2024-06-20T09:29:22.000Z,"[{'@type': 'Question', 'name': 'Le télétravail...",NaN,NaN,None,None
4,http://schema.org,JobPosting,NaN,2024-03-22T09:10:03Z,<p>We’re actively seeking a driven student wit...,INTERN,NaN,{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'BSPORT SOLU...","Application mobile, SaaS / Cloud Services","[{'@type': 'Place', 'address': {'@type': 'Post...",You will be a good fit to join us if you:Techn...,Data Analyst & Operations - Internship,2024-06-20T08:10:03.000Z,"[{'@type': 'Question', 'name': 'L'envoi d'un C...",NaN,NaN,None,None
5,http://schema.org,JobPosting,NaN,2024-03-20T15:37:52Z,"<p><img loading=""lazy"" width=""32""/>Le saviez-v...",INTERN,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Carrefour',...","Grande distribution, E-commerce, Grande consom...","[{'@type': 'Place', 'address': {'@type': 'Post...",Vous êtes titulaire d'un diplôme de niveau BAC...,Data Analyst Performance Opérationnelle (F/H) ...,2024-06-18T14:37:52.000Z,"[{'@type': 'Question', 'name': 'Le télétravail...",NaN,NaN,None,None
6,http://schema.org,JobPosting,NaN,2024-03-19T23:58:20Z,<p>Mazars souhaite chaleureusement son équipe ...,FULL_TIME,"{'@type': 'EducationalOccupationalCredential',...",{'@type': 'OccupationalExperienceRequirements'...,"{'@type': 'Organization', 'name': 'Mazars et l...",IT / Digital,"[{'@type': 'Place', 'address': {'@type': 'Post...","De formation Bac+5, vous êtes jeune diplômé(e)...",Data Analyst Junior - Paris - 2024 

In [36]:
!docker run --name posttest -d -p 5432:5432 -e POSTGRES_PASSWORD=fred postgres:alpine

8d4d7c5f84c2eb965df65c1a8864556c054afe45976bc9c736882f345e1da81b


Unable to find image 'postgres:alpine' locally
alpine: Pulling from library/postgres
4abcf2066143: Already exists
128d1b74d24d: Pulling fs layer
0a392327555d: Pulling fs layer
0e14a31643e8: Pulling fs layer
904e95badb7d: Pulling fs layer
8f6103a2e811: Pulling fs layer
02b106837f9f: Pulling fs layer
d34b010d3edc: Pulling fs layer
696a345da38f: Pulling fs layer
8f6103a2e811: Waiting
02b106837f9f: Waiting
d34b010d3edc: Waiting
696a345da38f: Waiting
904e95badb7d: Waiting
128d1b74d24d: Verifying Checksum
128d1b74d24d: Download complete
128d1b74d24d: Pull complete
0a392327555d: Verifying Checksum
0a392327555d: Pull complete
904e95badb7d: Verifying Checksum
904e95badb7d: Download complete
8f6103a2e811: Verifying Checksum
8f6103a2e811: Download complete
02b106837f9f: Verifying Checksum
02b106837f9f: Download complete
d34b010d3edc: Verifying Checksum
d34b010d3edc: Download complete
696a345da38f: Verifying Checksum
696a345da38f: Download complete
0e14a31643e8: Download complete
0e14a31643e8: Pul

In [57]:
from sqlalchemy import create_engine

# Database credentials
user = 'postgres'
password = 'fred'
host = 'localhost'  # or the IP if your PostgreSQL server is running elsewhere
port = '5432'       # default port for PostgreSQL used by our docker above
db = 'postgres'

# Create the connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [44]:
df['hiringOrganization'] = df['hiringOrganization'].apply(lambda x: x['name'])

In [45]:
df['jobLocation'] = df['jobLocation'].apply(lambda x: x[0]['address']['addressLocality'])

In [46]:
df = df.drop(['@context','baseSalary','educationRequirements','experienceRequirements','FAQPage'], axis=1)

In [48]:
df.head()

,@type,datePosted,description,employmentType,hiringOrganization,industry,jobLocation,qualifications,title,validThrough,minSalary,maxSalary,currency,salaryUnit
0,JobPosting,2024-03-25T15:00:02Z,<p><strong>Le Customer Data Analyst joue un rô...,INTERN,SPARTED,"SaaS / Cloud Services, Ressources humaines, Ed...",Paris,Tu es en étude au sein d’un cursus commercial ...,Customer Data Analyst (Stage - Juillet 2024) -...,2024-06-23T14:00:02.000Z,1000.0,1800.0,EUR,YEARLY
1,JobPosting,2024-03-25T08:44:58Z,<p><strong>Poste et missions</strong></p><p>La...,FULL_TIME,Caisse D'Épargne Languedoc-Roussillon,"Banque, Assurance, Finance",Montpellier,Issu(e) d'une formation Bac +5 types Ecole d'I...,Data Analyst - Direction Contrôle de Gestion -...,2024-06-23T07:44:58.000Z,NaN,NaN,None,None
2,JobPosting,2024-03-22T17:41:24Z,"<p>At Le Collectionist (LC), The Data team is ...",INTERN,Le Collectionist,"Hôtellerie, Tourisme, Immobilier particulier",Paris,Preferred backgroundBac+5 engineering or data ...,Data Analyst (H/F) - 6 months internship - Paris,2024-06-20T16:41:24.000Z,NaN,NaN,None,None
3,JobPosting,2024-03-22T10:29:22Z,<p><em><strong>Si dans votre métier vous aimez...,FULL_TIME,Bouygues Telecom,"IT / Digital, Objets connectés, Electronique /...",Meudon,À propos de vous :Vous êtes diplômé d'une form...,Data Analyst Financier F/H,2024-06-20T09:29:22.000Z,NaN,NaN,None,None
4,JobPosting,2024-03-22T09:10:03Z,<p>We’re actively seeking a driven student wit...,INTERN,BSPORT SOLUTION,"Application mobile, SaaS / Cloud Services",Paris,You will be a good fit to join us if you:Techn...,Data Analyst & Operations - Internship,2024-06-20T08:10:03.000Z,NaN,NaN,None,None


In [58]:
df.to_sql('job_table', engine, if_exists='replace', index=False)

60

In [59]:
import psycopg2

#connect to the PostgreSQL database
conn = psycopg2.connect(dbname='postgres', user=user, password=password, host=host, port=port)

#create a cursor object
cursor = conn.cursor()

#list all tables in the database
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table)

#check if job_table is created
if ('job_table',) in tables:
    print("\nTable 'job_table' exists.")
else:
    print("\nTable 'job_table' does not exist.")

#perform a verification query (e.g., selecting the first 5 rows)
query = "SELECT * FROM job_table LIMIT 5"
df = pd.read_sql(query, conn)
print("\nFirst 5 rows of 'job_table':")
print(df)


Tables in the database:
('job_table',)

Table 'job_table' exists.

First 5 rows of 'job_table':
        @type            datePosted  \
0  JobPosting  2024-03-25T15:00:02Z   
1  JobPosting  2024-03-25T08:44:58Z   
2  JobPosting  2024-03-22T17:41:24Z   
3  JobPosting  2024-03-22T10:29:22Z   
4  JobPosting  2024-03-22T09:10:03Z   

                                         description employmentType  \
0  <p><strong>Le Customer Data Analyst joue un rô...         INTERN   
1  <p><strong>Poste et missions</strong></p><p>La...      FULL_TIME   
2  <p>At Le Collectionist (LC), The Data team is ...         INTERN   
3  <p><em><strong>Si dans votre métier vous aimez...      FULL_TIME   
4  <p>We’re actively seeking a driven student wit...         INTERN   

                      hiringOrganization  \
0                                SPARTED   
1  Caisse D'Épargne Languedoc-Roussillon   
2                       Le Collectionist   
3                       Bouygues Telecom   
4                     

C:\Users\bertr\AppData\Local\Temp\ipykernel_24344\3776804960.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [61]:
!docker run -d --name example-mongo -p 27017:27017 mongo

3001532230dc74dd91ed9e975e27214eedc0172210882f274e3e074eb5942630


In [64]:
from pymongo import MongoClient

#connect to the MongoDB server (default is localhost on port 27017)
client = MongoClient('localhost', 27017)

#access the database (create it if it doesn't exist)
db = client['job_database']

#access the collection (similar to a table in relational databases)
collection = db['job_collection']

In [65]:
collection.insert_many(df.to_dict('records'))

InsertManyResult([ObjectId('6602d938eedde1281e213d5a'), ObjectId('6602d938eedde1281e213d5b'), ObjectId('6602d938eedde1281e213d5c'), ObjectId('6602d938eedde1281e213d5d'), ObjectId('6602d938eedde1281e213d5e')], acknowledged=True)

In [66]:
count = 0
for job in collection.find({"currency": "EUR"}):
    count+=1
print(count)

1
